In [ ]:
%pip install torch torchvision transformers datasets timm tf-keras




[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
zsh:1: no matches found: transformers[torch]
Note: you may need to restart the kernel to use updated packages.
zsh:1: command not found: pip
zsh:1: command not found: pip


In [13]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torch
torch.set_default_device(device='cuda:0')
torch.get_default_device() # device(type='cuda', index=0)

torch.__version__ # 2.3.0
# Chargement du processeur d'images et du modèle pré-entraîné
model_name = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(model_name)
model = AutoModelForImageClassification.from_pretrained(
    model_name,
    num_labels=10,  # Met le nombre de classes de ton dataset
    ignore_mismatched_sizes=True  # Utile si on change le nombre de classes
)


NameError: name '_C' is not defined

In [ ]:
from datasets import load_dataset
from torchvision import transforms
from PIL import Image

# Charger CIFAR-10 (ou un autre dataset)
dataset = load_dataset("cifar10")

# Transformer pour ViT
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Adapter la taille à ViT
    transforms.ToTensor(),
    transforms.Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
])

# Appliquer la transformation aux images
def transform_examples(examples):
    images = [transform(Image.fromarray(img)) for img in examples["img"]]
    labels = examples["label"]
    return {"pixel_values": images, "labels": labels}

# Appliquer le prétraitement au dataset
dataset = dataset.with_transform(transform_examples)


In [ ]:
from torch.utils.data import DataLoader

# Créer les DataLoaders pour l'entraînement et la validation
train_dataloader = DataLoader(dataset["train"], batch_size=32, shuffle=True)
val_dataloader = DataLoader(dataset["test"], batch_size=32)


In [ ]:
from transformers import TrainingArguments, Trainer

# Configuration de l'entraînement
training_args = TrainingArguments(
    output_dir="./vit-finetuned-cifar10",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,  # Nombre d'époques
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    push_to_hub=False  # Mettre True si tu veux sauvegarder sur Hugging Face
)

# Définition du trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=image_processor,  # Non nécessaire mais utile pour compatibilité
)

# Lancer l'entraînement
trainer.train()


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`

In [ ]:
model.save_pretrained("./vit-finetuned-cifar10")
image_processor.save_pretrained("./vit-finetuned-cifar10")


In [ ]:
from transformers import pipeline
import requests
from PIL import Image

# Charger le modèle entraîné
model_finetuned = AutoModelForImageClassification.from_pretrained("./vit-finetuned-cifar10")
classifier = pipeline("image-classification", model=model_finetuned, feature_extractor=image_processor)

# Charger une image de test
url = "https://upload.wikimedia.org/wikipedia/commons/9/9a/Cat_%28Felis_catus%29.jpg"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")

# Prédiction
predictions = classifier(image)
print(predictions)
